# Face movement detection

In [1]:
import os
import cv2
import numpy as np
from PIL import Image
import pickle
os.chdir('/Users/NidhiAneja/Documents/CodeDay/')
BASE_DIR = os.getcwd()
image_dir = os.path.join(BASE_DIR, "images")
face_cascade = cv2.CascadeClassifier('cascades/data/haarcascade_frontalface_alt2.xml') # CV MODEL
recognizer = cv2.face.LBPHFaceRecognizer_create() 
current_id = 0
label_ids = {}
y_labels = []
x_train = []
for root, dirs, files in os.walk(image_dir):
    for file in files:
        if file.endswith("png") or file.endswith("jpg"):
            path = os.path.join(root, file)
            label = os.path.basename(root).replace(" ", "-").lower()
            #print(label, path)
            if not label in label_ids:
                label_ids[label] = current_id
                current_id += 1
            id_ = label_ids[label]
            #print(label_ids)
            #y_labels.append(label) # some number
            #x_train.append(path) # verify this image, turn into a NUMPY arrray, GRAY
            pil_image = Image.open(path).convert("L") # grayscale
            size = (550, 550)
            final_image = pil_image.resize(size, Image.ANTIALIAS)
            image_array = np.array(final_image, "uint8")
            #print(image_array)
            faces = face_cascade.detectMultiScale(image_array, scaleFactor=1.5, minNeighbors=1)

            for (x,y,w,h) in faces:
                roi = image_array[y:y+h, x:x+w]
                x_train.append(roi)
                y_labels.append(id_)
with open(os.path.join(BASE_DIR, "pickles/face-labels.pickle"), 'wb') as f:
	pickle.dump(label_ids, f)
recognizer.train(x_train, np.array(y_labels))
recognizer.save(os.path.join(BASE_DIR, "recognizers/face-trainner.yml") )

FileNotFoundError: [WinError 3] The system cannot find the path specified: '/Users/NidhiAneja/Documents/CodeDay/'

# Run this this now

In [2]:
import numpy as np
import cv2
import pickle
import math
os.chdir('/Users/NidhiAneja/Documents/AI/Cafeteria/OpenCV-Python-Series-master/src')
BASE_DIR = os.getcwd()
face_cascade = cv2.CascadeClassifier(os.path.join(BASE_DIR,'cascades/data/haarcascade_frontalface_alt2.xml') )
eye_cascade = cv2.CascadeClassifier(os.path.join(BASE_DIR,'cascades/data/haarcascade_eye.xml') )
smile_cascade = cv2.CascadeClassifier(os.path.join(BASE_DIR, 'cascades/data/haarcascade_smile.xml'))
recognizer = cv2.face.LBPHFaceRecognizer_create()
#recognizer.read(os.path.join(BASE_DIR, 'recognizers/face-trainner.yml') )
recognizer.read(os.path.join(BASE_DIR, "recognizers/face-trainner.yml") )
labels = {"person_name": 1}
with open("pickles/face-labels.pickle", 'rb') as f:
	og_labels = pickle.load(f)
	labels = {v:k for k,v in og_labels.items()}
lstx = []
lsty = []
cap = cv2.VideoCapture(0)
while(True):
    # Capture frame-by-frame
    test = 0
    ret, frame = cap.read()
    gray  = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.5, minNeighbors=5)
    for (x, y, w, h) in faces:
    	#print(x,y,w,h)
    	roi_gray = gray[y:y+h, x:x+w] #(ycord_start, ycord_end)
    	roi_color = frame[y:y+h, x:x+w]
    	# recognize? deep learned model predict keras tensorflow pytorch scikit learn
    	id_, conf = recognizer.predict(roi_gray)
    	if conf>=25:
    		#print(5: #id_)
    		#print(labels[id_])
    		font = cv2.FONT_HERSHEY_SIMPLEX
    		name = 'The player'
    		color = (0, 0, 255)
    		stroke = 3
    		cv2.putText(frame, name, (x,y), font, 1.5, color, stroke, cv2.LINE_AA)
    	img_item = "the_image_for_the_model.png"
    	cv2.imwrite(img_item, roi_color)
    	color = (255, 0, 0) #BGR 0-255 
    	stroke = 2
    	end_cord_x = x + w
    	end_cord_y = y + h
    	cv2.rectangle(frame, (x, y), (end_cord_x, end_cord_y), color, stroke)
    	lstx.append(x)
    	lsty.append(y)
    	#subitems = smile_cascade.detectMultiScale(roi_gray)
    	#for (ex,ey,ew,eh) in subitems:
    	#	cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)
    	with open("Names.txt", "w+") as myfile:
            myfile.write("I detect " + (labels[id_]) + ".\n")
    	#labels[id_] = the_name_detect        
        
    # Display the resulting frame    
    black = '/Users/NidhiAneja/Documents/CodeDay/Other/black.png'
    #cv2.imshow('frame',frame)
    cv2.imshow('Nothing',frame)
    #FIX SO PRINTS ALL OF THE NAMES
    '''output = open("Names.txt","w+")
    output.write("I detect " + (labels[id_]) + ".\n")'''
    if cv2.waitKey(2) & 0xFF == ord('q'):
        #output.close()
        break        

cap.release()
cv2.destroyAllWindows()
#print(lstx)
#print(lsty)
firstx = lstx[0]
lastx = lstx[-1]
firsty = lsty[0]
lasty = lsty[-1]

totalmovx = lastx - firstx
totalmovy = lasty - firsty
movxfin = totalmovx * totalmovx
movyfin = totalmovy * totalmovy
total = math.sqrt(movxfin + movxfin)
total_inchs = total/30
#30 pixels is about 1 inch 
print('You moved') 
print(total)
print('pixels.')
print('Which is about ')
print(total_inchs)
print('inches.')


if total <= 50:
    print('You were not so scared. Good job')
elif total > 50 and total < 100:
    print('You got somewhat scared. We sort of got you!')
elif total >=100 and total < 150:
    print('You got scared. We got you.')
else:
    print('You were terrified. We got you good. :)')
    
    


You moved
492.1463197058371
pixels.
Which is about 
16.404877323527902
inches.
You were terrified. We got you good. :)
